In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/MalawiWoodfuelSite_1a')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
# Añadir Google Satellite como mapa base
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [3]:
roi_feature = ee.FeatureCollection(roi).first()

# Extraer la geometría de la característica
roi_geometry = roi_feature.geometry()

# Obtener las coordenadas de la geometría
roi_coordinates = roi_geometry.coordinates().getInfo()

# Imprimir las coordenadas para verificar
print("Coordenadas de ROI:", roi_coordinates)

# Guardar las coordenadas en una variable
roi_coords = roi_coordinates

ee_roi =  ee.Geometry.Polygon(roi_coords) 

Coordenadas de ROI: [[[35.147307857226416, -15.036316131901854], [35.15253608286859, -15.223952919778808], [35.15776440008984, -15.411589648079637], [35.20994926360607, -15.410686693842585], [35.26213417872549, -15.409783744326894], [35.314318994087046, -15.408880778443212], [35.366503893152895, -15.407977790870213], [35.3589679641149, -15.032891601485606], [35.30605294737285, -15.033747715996299], [35.25313786054075, -15.034603909691883], [35.20022286170109, -15.035460051719076], [35.147307857226416, -15.036316131901854]]]


In [4]:
# Definir el año de análisis
year = 2017

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

In [5]:
# Filtrar la colección Sentinel-2
s2Col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Filtrar la colección Sentinel-2 cloud score
cloudCol = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Combinar la colección, máscara de nubes y hacer un composite mediano
def mask_clouds(image):
    cloud_score = ee.Image(cloudCol.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

# Aplicar la función de enmascaramiento a la colección
s2Masked = s2Col.map(mask_clouds)

# Calcular la composición mediana
image = s2Masked.median().toFloat().clip(roi)

# Calcular NDVI, MNDWI y NDBI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')


evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
    }
).rename('EVI')


savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('B8'),
        'RED': image.select('B4')
    }
).rename('SAVI')

# Añadir bandas NDVI, MNDWI, NDBI, EVI y SAVI a la imagen
image = image.addBands([ndvi, mndwi, ndbi, evi, savi])

# Integrar la banda "label" de Dynamic World
dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dwLabel = dwCol.select('label').median().clip(roi)

# Añadir la banda "label" de Dynamic World a la imagen
image = image.addBands(dwLabel.rename('label'))

# Cargar la imagen de altura de dosel
canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")

# Recortar la imagen de altura de dosel a la ROI
canopy_height = canopy_height.clip(roi)

# Agregar la banda de altura de dosel a la imagen de datos de biomasa de GEDI
image = image.addBands(canopy_height.rename('CH'))


# Función de preprocesamiento
def preprocess(image):
    # Aplicar corrección de ángulo de incidencia
    # image = apply_incidence_angle_correction(image)
    # Filtrado de Speckle
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')

    return speckle_filtered
# Definir la función de máscara de borde
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# Filtrar y procesar la colección Sentinel-1 para VV
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate(start, end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')

    .map(mask_edge)
    .map(preprocess)
)

# Filtrar y procesar la colección Sentinel-1 para VH
img_vh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate(start, end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')

    .map(mask_edge)
    .map(preprocess)
)


# Promedio de imágenes VV y VH
s1_vv = img_vv.median().rename('VV')
s1_vh = img_vh.median().rename('VH')



# Calcular el índice RVI
rvi = s1_vh.divide(s1_vv.add(s1_vh)).multiply(4).rename('RVI')
vv_plus_vh = s1_vv.add(s1_vh).rename('VV_plus_VH')
vv_minus_vh = s1_vv.subtract(s1_vh).rename('VV_minus_VH')
vv_div_vh = s1_vv.divide(s1_vh).rename('VV_div_VH')
vv_vh_div_2 = s1_vv.add(s1_vh).divide(2).rename('VV_VH_div_2')

# Combinar las bandas VV, VH y RVI en una sola imagen
s1_combined = ee.Image.cat([s1_vv, s1_vh, vv_plus_vh, vv_minus_vh, vv_div_vh, vv_vh_div_2, rvi]).clip(roi)

# Añadir las bandas VV, VH, RVI y álgebra de bandas a la imagen
image = image.addBands(s1_combined)

# Añadir SRTM DEM y pendiente
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

# Añadir las bandas DEM y pendiente a la imagen
image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)

# Filtrar y procesar la colección PALSAR-2 para HH
img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')
    .map(preprocess)
)

# Filtrar y procesar la colección PALSAR-2 para HV
img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')
    .map(preprocess)
)

# Promedio de imágenes HH y HV
palsar_hh = img_hh.median().rename('HH')
palsar_hv = img_hv.median().rename('HV')

# Calcular el índice RVI
rvi = palsar_hv.divide(palsar_hh.add(palsar_hv)).multiply(4).rename('RVI_palsar')
hh_plus_hv = palsar_hh.add(palsar_hv).rename('HH_plus_HV')
hh_minus_hv = palsar_hh.subtract(palsar_hv).rename('HH_minus_HV')
hh_div_hv = palsar_hh.divide(palsar_hv).rename('HH_div_HV')
hh_hv_div_2 = palsar_hh.add(palsar_hv).divide(2).rename('HH_HV_div_2')


# Combinar las bandas HH, HV y RVI en una sola imagen
palsar_combined = ee.Image.cat([palsar_hh, palsar_hv, hh_plus_hv, hh_minus_hv, hh_div_hv, hh_hv_div_2, rvi]).clip(roi)

# Añadir las bandas HH, HV, RVI y álgebra de bandas a la imagen final
image = image.addBands(palsar_combined)


bands = ['B11', 'B12','B8', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','RVI', 'VH', 'VV', 'VV_VH_div_2', 'VV_div_VH', 'VV_minus_VH', 'VV_plus_VH', 'elevation', 'slope', 'HH', 'HV','RVI_palsar','HH_plus_HV','HH_minus_HV','HH_div_HV','HH_HV_div_2','label']
# Seleccionar solo las bandas especificadas
image = image.select(bands)


In [6]:
# Parámetros de visualización para Sentinel-1
visParamsS1 = {
    'min': -25,
    'max': 0,
    'bands': ['VV']
}


Map.centerObject(roi, 10)
Map.addLayer(image, visParamsS1 , 'VV', False)
Map

EEException: Image.select: Band pattern 'B4' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands

In [43]:
feature_names = ['B11', 'B12','B8', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','RVI', 'VH', 'VV', 'VV_VH_div_2', 'VV_div_VH', 'VV_minus_VH', 'VV_plus_VH', 'elevation', 'slope', 'HH', 'HV','RVI_palsar','HH_plus_HV','HH_minus_HV','HH_div_HV','HH_HV_div_2']
label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/RF_Malawi"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [66]:
feature_names = ['B11', 'B12','B8', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI','EVI', 'SAVI','RVI', 'VH', 'VV', 'VV_VH_div_2', 'VV_div_VH', 'VV_minus_VH', 'VV_plus_VH', 'elevation', 'slope', 'HH', 'HV','RVI_palsar','HH_plus_HV','HH_minus_HV','HH_div_HV','HH_HV_div_2']
label = "agbd"
user_id = 'users/facuboladerasgee'
def recombine_model_parts(user_id, base_asset_id, num_parts):
    parts = [ee.FeatureCollection(f"{base_asset_id}_part_{i}") for i in range(num_parts)]
    combined_fc = ee.FeatureCollection(parts[0])
    
    for part in parts[1:]:
        combined_fc = combined_fc.merge(part)
    
    return ml.fc_to_classifier(combined_fc)

# Ejemplo de uso:
num_parts = 2  # Ajusta este número según la cantidad de partes exportadas
base_asset_id = f"{user_id}/Rf_Malawi_complete"
recombined_classifier = recombine_model_parts(user_id, base_asset_id, num_parts)

# Clasificar la imagen con el clasificador recombinado
classified = image.select(feature_names).classify(recombined_classifier)


In [14]:
# # Reducir la imagen clasificada a su valor máximo en la región de interés
# max_value = classified.reduceRegion(
#     reducer=ee.Reducer.max(),
#     geometry=roi,
#     scale=100,
#     maxPixels=1e9
# )

# # Obtener el valor máximo de la reducción
# max_agbd = max_value.get('classification').getInfo()
# print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 155.93490694999983


In [67]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 200,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [68]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='Malawi_2021_complete',
    folder='Estimators_Malawi_complete',
    region=ee_roi,
    scale=30,
    crs='EPSG:4326',  # CRS adecuado para Nepal
    
    
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
